In [1]:
import pandas as pd
from sqlalchemy import create_engine
import geojson

In [2]:
from config import user, password

In [3]:
import pymysql
pymysql.install_as_MySQLdb()
engine = create_engine('mysql://' + user + ':' + password + '@imdb-project.cj1m13sfkxlg.us-east-2.rds.amazonaws.com:3306/gundata')


In [4]:
#Query records
data = engine.execute("SELECT Year(Date), COALESCE(state, 'Total') as st, SUM(n_killed) as Gun_Deaths,latitude as lat,longitude as lon FROM gundata GROUP BY Year(Date),st WITH ROLLUP")
query1 = "SELECT Year(Date), COALESCE(state, 'Total') as st, SUM(n_killed) as Gun_Deaths,latitude as lat,longitude as lon FROM gundata GROUP BY Year(Date),st WITH ROLLUP"

In [5]:
for record in data:
    print(record)


(2013, 'Alabama', Decimal('3'), 34.7982, -87.6854)
(2013, 'Arizona', Decimal('8'), 33.5985, -112.113)
(2013, 'California', Decimal('46'), 33.909, -118.333)
(2013, 'Colorado', Decimal('4'), 39.6518, -104.802)
(2013, 'Connecticut', Decimal('2'), 41.7957, -72.6635)
(2013, 'Delaware', Decimal('3'), 39.7407, -75.5499)
(2013, 'District of Columbia', Decimal('12'), 38.8978, -76.9717)
(2013, 'Florida', Decimal('22'), 26.6968, -80.6837)
(2013, 'Georgia', Decimal('5'), 32.826, -83.6704)
(2013, 'Illinois', Decimal('14'), 41.7592, -87.5628)
(2013, 'Indiana', Decimal('9'), 39.7927, -86.0328)
(2013, 'Kansas', Decimal('6'), 37.7044, -97.3289)
(2013, 'Kentucky', Decimal('4'), 38.0485, -84.453)
(2013, 'Louisiana', Decimal('8'), 29.9435, -90.0836)
(2013, 'Maryland', Decimal('5'), 39.2899, -76.6412)
(2013, 'Michigan', Decimal('9'), 42.9371, -85.6853)
(2013, 'Minnesota', Decimal('3'), 44.9553, -93.2489)
(2013, 'Mississippi', Decimal('2'), 32.2649, -90.2141)
(2013, 'Missouri', Decimal('9'), 38.7067, -90.24

In [6]:
df1 = pd.read_sql_query(query1,engine)


In [7]:
df1.to_json(path_or_buf='overall_gun_deaths_by_year.json',orient='records')

In [8]:
query2 = """
SELECT year(gundata.date),gundata.state,city_or_county,latitude,longitude,sum(n_killed) FROM gundata
LEFT JOIN		(SELECT DISTINCT state
				FROM gundata
				GROUP BY state
				ORDER BY SUM(n_killed) DESC
                LIMIT 5) as sub
ON gundata.state = sub.state
WHERE sub.state IS NOT NULL
GROUP BY city_or_county, year(gundata.date)
ORDER BY sum(n_killed) DESC;
"""


df2 = pd.read_sql_query(query2,engine)
df2.to_json(path_or_buf='gun_deaths in top 5 states.json',orient='records')


In [9]:
#Used Stored Procedure
df3 = pd.read_sql_query("call Cause_Of_Gun_Deaths()", engine)
df3.to_json(path_or_buf='Cause_of_Gun_Deaths_by_City_State_Year.json',orient='records')
